# Minimal Database Mapping with SQLAlchemy

You don't always need a framework-integrated database solution. Sometimes, the simpler solution offers better performance with less resources.

Modern frameworks often come with an in-house database modeling solution; such as Laravel's [Eloquent](https://laravel.com/docs/9.x/eloquent), Rails' [Active Record](https://guides.rubyonrails.org/active_record_basics.html), or Django's [Model](https://docs.djangoproject.com/en/4.1/topics/db/models/). While convenient, these ORM solutions aren't always the most efficeient design; especially if used within environments of complex schemas and/or joined database entities. In these situations, as with simple or non-framework applications, a domain language toolkit such as SQLAlchemy could be worth consideration.

## Weather Feed

Let's consider a simple application capable of storing weather data from various hardware devices. These "feeders" will all send a timestamped data packet which our applicaiton will record into a relational database store. We'll use the application to provide generic reports. The implementation of this application, as well as the feeders, is outside the scope of this post, but let's consider how SQLAlchemy could be used for database support.

## Start Your Engine

SQLAlchemy provides two foundational solutions: a Core toolkit which provides database engine and query support, and an ORM extension to provide python mappings between database entities and application-defined objects. We'll begin with a solution using only the Core toolkit, then show how to build on this with ORM objects.

To connect with our database, we first need to configure the Engine: a resource pool in our global namespace which can be used to provide local DBAPI connections throughout the rest of our application. To keep the example simple, we'll configure an in-memory SQLite instance.

In [68]:
import sqlalchemy as sa
engine = sa.create_engine("sqlite:///temps.db", echo=True, future=True)

Note: The optional flag `create_engine.echo` provides generic debug logging of generated SQL commands, greatly improving development feedback for local scripting. The `create_engine.future` flag gives us access to upcoming 2.0 syntax within SQLAlchemy. This is primarily of interest to existing sites looking to move from 1.x to 2.x API calls.

## Schema Catalog

We next need to define our database schema within a SQLAlchemy `MetaData` catalog; similar to our `engine` resource pool, this will be a global object within our application, and allows us to interact with all database entities via the `engine` pool.

In [69]:
metadata = sa.MetaData()

feeders = sa.Table(
    "feeders",
    metadata,
    sa.Column("id", sa.Integer, primary_key=True),
    sa.Column("name", sa.Text),
    sa.Column("lat", sa.Float),
    sa.Column("long", sa.Float)
)

temps = sa.Table(
    "temps",
    metadata,
    sa.Column("id", sa.Integer, primary_key=True),
    sa.Column("feeder_id", sa.ForeignKey("feeders.id")),
    sa.Column("ts", sa.DateTime, nullable=False),
    sa.Column("celsius", sa.Float),
    sa.Column("condition", sa.Text())
)

With our schemas defined in code, we can generate engine-specific DDL creation statements using the following command:

In [ ]:
metadata.create_all(engine)

```sql
BEGIN (implicit)
PRAGMA main.table_info("feeders")
...
PRAGMA main.table_info("temps")
...
CREATE TABLE feeders (
	id INTEGER NOT NULL, 
	name TEXT, 
	lat FLOAT, 
	long FLOAT, 
	PRIMARY KEY (id)
)
...
CREATE TABLE temps (
	id INTEGER NOT NULL, 
	feeder_id INTEGER, 
	ts DATETIME NOT NULL, 
	celsius FLOAT, 
	condition TEXT, 
	PRIMARY KEY (id), 
	FOREIGN KEY(feeder_id) REFERENCES feeders (id)
)
...
COMMIT
```

For example, our application might have a dedicated function to receive feeder data packets as such:

In [71]:
def store_packet(feeder_id, data):
    with engine.begin() as conn:
        conn.execute(temps.insert(), dict(feeder_id=feeder_id, **data))
        # implicit conn.commit() if no error

In [ ]:
from datetime import datetime

store_packet(1, dict(ts=datetime.now(), celsius=24.5, condition="overcast"))

## Reporting via Query

In [74]:
def low_by_date(feeder, ts_start, ts_end=None):
    with engine.connect() as conn:
        query = sa.select(sa.func.min(temps.c.celsius)).where(
            temps.c.feeder_id == feeder,
            temps.c.ts.between(ts_start, ts_end or datetime.now())
        )

        return conn.execute(query).scalar()

low_by_date(1, datetime(2022, 12, 19))
# 24.0

2022-12-19 15:59:24,114 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-19 15:59:24,114 INFO sqlalchemy.engine.Engine SELECT min(temps.celsius) AS min_1 
FROM temps 
WHERE temps.feeder_id = ? AND temps.ts BETWEEN ? AND ?
2022-12-19 15:59:24,114 INFO sqlalchemy.engine.Engine [cached since 66.6s ago] (1, '2022-12-19 00:00:00.000000', '2022-12-19 15:59:24.113971')
2022-12-19 15:59:24,115 INFO sqlalchemy.engine.Engine ROLLBACK


24.0

```sql
BEGIN (implicit)
SELECT min(temps.celsius) AS min_1
FROM temps
WHERE temps.feeder_id = ? AND temps.ts BETWEEN ? AND ?
...
ROLLBACK
```